Send different subsets of input features through the wide or deep paths.



* We will send 5 features (features 0 to 4), and 6 through the deep path (features 2 to 7).

* 3 features will go through both (features 2, 3 and 4).


Then, visualize with TensorBoard.

In [ ]:
# import libraries

import numpy as np
import tensorflow as tf
from tensorflow import keras
import os

In [ ]:
np.random.seed(42) # set this seed as the last two digits of your student ID
tf.random.set_seed(42) # set this seed as the last two digits of your student ID

In [ ]:
# import california_housing dataset

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

# split dataset
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) # X_train.shape->(11610, 8).  8 features
X_valid = scaler.transform(X_valid)


In [ ]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

In [ ]:
root_logdir = os.path.join(os.curdir, "my_logs")

In [ ]:
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
run_logdir